## Segmenting Liver 

#### *Table of Contents*
1. [NIfTI files](#chapter1)
2. [Images preprocessing](#chapter2)
2. [Model architecture](#chapter3)
3. [Training](#chapter4)
4. [Testing and metrics](#chapter5)

- libraries:

In [12]:
import os
import shutil
from glob import glob
import nibabel as nib
import pydicom
import numpy as np
import dicom2nifti

- Converting NIfTI files to DICOM

In [6]:
def nifti_to_dicom(nifti_path, output_path):
    """Converts a NIfTI file to a series of DICOM files in a separate folder.

    Args:
        nifti_path (str): Path to the NIfTI file.
        output_path (str): Output directory for DICOM files.
    """

    # Extract patient ID from nifti file name
    patient_id = os.path.basename(nifti_path).split('.')[0]
    patient_output_path = os.path.join(output_path, patient_id)

    # Create patient output directory
    os.makedirs(patient_output_path, exist_ok=True)

    # Load NIfTI data
    img = nib.load(nifti_path)
    data = img.get_fdata()

    # Basic DICOM metadata (replace with actual values)
    ds = pydicom.Dataset()
    ds.PatientID = patient_id
    ds.StudyInstanceUID = pydicom.uid.generate_uid()
    ds.SeriesInstanceUID = pydicom.uid.generate_uid()
    ds.Modality = 'MR'  # Replace with appropriate modality
    ds.PixelSpacing = [0.5, 0.5]  # Replace with actual pixel spacing
    ds.SliceThickness = 3.0  # Replace with actual slice thickness

    # Set is_little_endian and is_implicit_VR
    ds.is_little_endian = True  # Assuming little-endian byte order (adjust if needed)
    ds.is_implicit_VR = False  # Assuming explicit VR (adjust if needed)

    # Iterate over slices
    for i in range(data.shape[2]):
        slice_data = data[:, :, i]

        # Create DICOM dataset for the slice
        ds.ImageInstanceUID = pydicom.uid.generate_uid()
        ds.InstanceNumber = i + 1
        ds.SliceLocation = -(i * ds.SliceThickness)  # Adjust based on slice orientation

        # Convert image data to appropriate format (e.g., int16)
        slice_data = slice_data.astype(np.int16)

        # Set pixel data
        ds.PixelData = slice_data.tobytes()
        ds.BitsAllocated = 16
        ds.SamplesPerPixel = 1
        ds.HighBit = 15
        ds.PixelRepresentation = 0

        # Write DICOM file
        output_file = os.path.join(patient_output_path, f'image_{i+1}.dcm')
        pydicom.filewriter.dcmwrite(output_file, ds)

# IMAGES TRANSFORMATION
nifti_folder = '../data/Task03_Liver/Task03_Liver/images'

output_folder = '../data/Task03_Liver/Task03_Liver/dicom_data/dicom_images'

# Iterate over NIfTI files
for nifti_file in glob(os.path.join(nifti_folder, '*.nii.gz')):
    nifti_to_dicom(nifti_file, output_folder)

c:\Users\adaml\anaconda3\Lib\site-packages\pydicom\dataset.py:2710: UserWarning: Camel case attribute 'ImageInstanceUID' used which is not in the element keyword data dictionary
  warn_and_log(msg)


In [7]:
# LABELS TRANSFORMATION
nifti_folder_labels = '../data/Task03_Liver/Task03_Liver/labels'

output_folder_labels = '../data/Task03_Liver/Task03_Liver/dicom_data/dicom_labels'

# Iterate over NIfTI files
for nifti_file in glob(os.path.join(nifti_folder_labels, '*.nii.gz')):
    nifti_to_dicom(nifti_file, output_folder_labels)

c:\Users\adaml\anaconda3\Lib\site-packages\pydicom\dataset.py:2710: UserWarning: Camel case attribute 'ImageInstanceUID' used which is not in the element keyword data dictionary
  warn_and_log(msg)


In [8]:
input_labels = "../data/Task03_Liver/Task03_Liver/dicom_data/dicom_labels"
output_groups_labels = "../data/Task03_Liver/Task03_Liver/dicom_groups/dicom_group_label"

In [9]:
input_images = "../data/Task03_Liver/Task03_Liver/dicom_data/dicom_images"
output_groups_images = "../data/Task03_Liver/Task03_Liver/dicom_groups/dicom_group_image"

- Creating groups of 64 slices:

 (becasue number of slices can be slightly different for every person - we need the same number of slices as input)

In [10]:
INT_SLICES_NUM = 64

# LABELS

for person in sorted(glob(input_labels + '/*')):
    person_name = os.path.basename(os.path.normpath(person))
    num_folders = int(len(sorted(glob(person + '/*'))) / INT_SLICES_NUM)   

    # GROUPS
    for i in range (num_folders):
        output_path = os.path.join(output_groups_labels, person_name + '_' + str(i))
        os.mkdir(output_path)

        # SLICES
        for j, file in enumerate(sorted(glob(person + '/*'))):
            if j > INT_SLICES_NUM - 1:
                break
            shutil.move(file, output_path)

In [11]:
# IMAGES

for person in sorted(glob(input_images + '/*')):
    person_name = os.path.basename(os.path.normpath(person))
    num_folders = int(len(sorted(glob(person + '/*'))) / INT_SLICES_NUM)   

    # GROUPS
    for i in range (num_folders):
        output_path = os.path.join(output_groups_images, person_name + '_' + str(i))
        os.mkdir(output_path)

        # SLICES
        for j, file in enumerate(sorted(glob(person + '/*'))):
            if j > INT_SLICES_NUM - 1:
                break
            shutil.move(file, output_path)

- convertion from DICOM to NIfTI

In [22]:
# groups of 64 slices 
dicom_groups_labels = "../data/Task03_Liver/Task03_Liver/dicom_groups/dicom_group_label/*"
dicom_groups_images = "../data/Task03_Liver/Task03_Liver/dicom_groups/dicom_group_image/*"

In [23]:
images_list = sorted(glob(dicom_groups_images))
labels_list = sorted(glob(dicom_groups_labels))

In [24]:
# final NIfTI
nifti_images = "../data/Task03_Liver/Task03_Liver/nifti_data/nifti_images"
nifti_labels = "../data/Task03_Liver/Task03_Liver/nifti_data/nifti_labels"

In [28]:
# Check if the directory contains valid DICOM files
def is_dicom_directory_valid(directory):
    dicom_files = [f for f in os.listdir(directory) if f.endswith('.dcm')]
    if not dicom_files:
        return False

    for dicom_file in dicom_files:
        try:
            pydicom.dcmread(os.path.join(directory, dicom_file))
        except:
            return False
    return True

In [27]:
for person in images_list:
    person_name = os.path.basename(os.path.normpath(person))
    print(f"Processing: {person_name}")
    
    dicom_list = dicom2nifti.common.read_dicom_directory(person)
    print(f"Loaded {len(dicom_list)} DICOM files from {person}")


    # Check if the directory contains valid DICOM files
    if is_dicom_directory_valid(person):
        try:
            dicom2nifti.dicom_series_to_nifti(person, os.path.join(nifti_images, person_name + ".nii.gz"))
            print(f"Successfully converted {person_name}")
        except Exception as e:
            print(f"Error processing {person_name}: {str(e)}")
    else:
        print(f"No valid DICOM files found in {person}")


liver_0_0
../data/Task03_Liver/Task03_Liver/dicom_groups/dicom_group_image\liver_0_0


IndexError: list index out of range

In [ ]:
for person in labels_list:
    person_name = os.path.basename(os.path.normpath(person))
    dicom2nifti.dicom_series_to_nifti(person, os.path.join(nifti_labels + "_" + person_name + ".nii"))